# 数据分析示例

这个Notebook展示了如何使用Jupyter进行数据分析，包括加载历史数据、计算技术指标和可视化数据。

## 1. 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 设置绘图风格
plt.style.use('ggplot')
sns.set_style('whitegrid')
%matplotlib inline

# 导入vnpy相关模块
from vnpy.trader.constant import Interval
from vnpy.trader.database import database_manager
from vnpy.trader.utility import BarGenerator, ArrayManager

## 2. 加载历史数据

从数据库加载历史K线数据，并转换为DataFrame格式。

In [ ]:
# 设置数据参数
symbol = "BTCUSDT"
exchange = "BINANCE"
interval = Interval.MINUTE
start = datetime(2023, 1, 1)
end = datetime(2023, 12, 31)

# 从数据库加载历史数据
bars = database_manager.load_bar_data(
    symbol=symbol,
    exchange=exchange,
    interval=interval,
    start=start,
    end=end
)

# 转换为DataFrame
data = []
for bar in bars:
    data.append({
        "datetime": bar.datetime,
        "open": bar.open_price,
        "high": bar.high_price,
        "low": bar.low_price,
        "close": bar.close_price,
        "volume": bar.volume
    })

df = pd.DataFrame(data)
df.set_index("datetime", inplace=True)

# 显示数据
df.head()

## 3. 数据预处理

对数据进行预处理，包括处理缺失值、重采样等。

In [ ]:
# 检查缺失值
print(f"缺失值数量:\n{df.isnull().sum()}")

# 处理缺失值
df = df.dropna()

# 重采样为日K线
daily_df = df.resample('D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})

# 删除缺失值
daily_df = daily_df.dropna()

# 显示日K线数据
daily_df.head()

## 4. 计算技术指标

计算常用的技术指标，如移动平均线、MACD、RSI等。

In [ ]:
# 计算移动平均线
daily_df["ma5"] = daily_df["close"].rolling(5).mean()
daily_df["ma10"] = daily_df["close"].rolling(10).mean()
daily_df["ma20"] = daily_df["close"].rolling(20).mean()
daily_df["ma60"] = daily_df["close"].rolling(60).mean()

# 计算MACD
def calculate_macd(df, fast=12, slow=26, signal=9):
    df = df.copy()
    df["ema_fast"] = df["close"].ewm(span=fast, adjust=False).mean()
    df["ema_slow"] = df["close"].ewm(span=slow, adjust=False).mean()
    df["macd"] = df["ema_fast"] - df["ema_slow"]
    df["signal"] = df["macd"].ewm(span=signal, adjust=False).mean()
    df["histogram"] = df["macd"] - df["signal"]
    return df

daily_df = calculate_macd(daily_df)

# 计算RSI
def calculate_rsi(df, window=14):
    df = df.copy()
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    df["rsi"] = 100 - (100 / (1 + rs))
    return df

daily_df = calculate_rsi(daily_df)

# 计算布林带
def calculate_bollinger_bands(df, window=20, num_std=2):
    df = df.copy()
    df["middle_band"] = df["close"].rolling(window=window).mean()
    df["std"] = df["close"].rolling(window=window).std()
    df["upper_band"] = df["middle_band"] + (df["std"] * num_std)
    df["lower_band"] = df["middle_band"] - (df["std"] * num_std)
    return df

daily_df = calculate_bollinger_bands(daily_df)

# 显示计算后的数据
daily_df.tail()

## 5. 数据可视化

可视化价格和技术指标，帮助分析市场趋势和模式。

In [ ]:
# 绘制价格和移动平均线
plt.figure(figsize=(12, 6))
plt.plot(daily_df.index, daily_df["close"], label="Close")
plt.plot(daily_df.index, daily_df["ma5"], label="MA5")
plt.plot(daily_df.index, daily_df["ma20"], label="MA20")
plt.plot(daily_df.index, daily_df["ma60"], label="MA60")
plt.title(f"{symbol} Price and Moving Averages")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 绘制布林带
plt.figure(figsize=(12, 6))
plt.plot(daily_df.index, daily_df["close"], label="Close")
plt.plot(daily_df.index, daily_df["middle_band"], label="Middle Band")
plt.plot(daily_df.index, daily_df["upper_band"], label="Upper Band")
plt.plot(daily_df.index, daily_df["lower_band"], label="Lower Band")
plt.fill_between(daily_df.index, daily_df["upper_band"], daily_df["lower_band"], alpha=0.1)
plt.title(f"{symbol} Bollinger Bands")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 绘制MACD
plt.figure(figsize=(12, 8))

# 价格子图
plt.subplot(3, 1, 1)
plt.plot(daily_df.index, daily_df["close"], label="Close")
plt.title(f"{symbol} Price")
plt.legend()
plt.grid(True)

# MACD子图
plt.subplot(3, 1, 2)
plt.plot(daily_df.index, daily_df["macd"], label="MACD")
plt.plot(daily_df.index, daily_df["signal"], label="Signal")
plt.bar(daily_df.index, daily_df["histogram"], label="Histogram")
plt.title("MACD")
plt.legend()
plt.grid(True)

# RSI子图
plt.subplot(3, 1, 3)
plt.plot(daily_df.index, daily_df["rsi"], label="RSI")
plt.axhline(y=70, color="r", linestyle="--")
plt.axhline(y=30, color="g", linestyle="--")
plt.title("RSI")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 6. 统计分析

对价格和收益率进行统计分析，了解市场的统计特性。

In [ ]:
# 计算日收益率
daily_df["returns"] = daily_df["close"].pct_change()

# 描述性统计
print("收益率描述性统计:")
print(daily_df["returns"].describe())

# 绘制收益率分布
plt.figure(figsize=(12, 6))
sns.histplot(daily_df["returns"].dropna(), kde=True)
plt.title(f"{symbol} Daily Returns Distribution")
plt.xlabel("Returns")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

# 绘制收益率QQ图
from scipy import stats
plt.figure(figsize=(12, 6))
stats.probplot(daily_df["returns"].dropna(), plot=plt)
plt.title(f"{symbol} Daily Returns Q-Q Plot")
plt.grid(True)
plt.show()

## 7. 相关性分析

分析不同交易品种之间的相关性，寻找交易机会。

In [ ]:
# 加载其他交易品种的数据
symbols = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "ADAUSDT", "DOGEUSDT"]
correlation_data = {}

for sym in symbols:
    # 从数据库加载历史数据
    bars = database_manager.load_bar_data(
        symbol=sym,
        exchange=exchange,
        interval=Interval.DAILY,
        start=start,
        end=end
    )
    
    # 转换为DataFrame
    data = []
    for bar in bars:
        data.append({
            "datetime": bar.datetime,
            "close": bar.close_price
        })
    
    sym_df = pd.DataFrame(data)
    sym_df.set_index("datetime", inplace=True)
    
    # 存储收盘价
    correlation_data[sym] = sym_df["close"]

# 创建包含所有交易品种收盘价的DataFrame
correlation_df = pd.DataFrame(correlation_data)

# 计算收益率
returns_df = correlation_df.pct_change().dropna()

# 计算相关性矩阵
correlation_matrix = returns_df.corr()

# 绘制相关性热图
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Cryptocurrency Returns Correlation Matrix")
plt.show()

## 8. 交易信号生成

基于技术指标生成交易信号，为策略开发提供基础。

In [ ]:
# 生成移动平均线交叉信号
daily_df["ma_cross_signal"] = 0
daily_df.loc[daily_df["ma5"] > daily_df["ma20"], "ma_cross_signal"] = 1  # 多头信号
daily_df.loc[daily_df["ma5"] < daily_df["ma20"], "ma_cross_signal"] = -1  # 空头信号

# 生成MACD信号
daily_df["macd_signal"] = 0
daily_df.loc[daily_df["macd"] > daily_df["signal"], "macd_signal"] = 1  # 多头信号
daily_df.loc[daily_df["macd"] < daily_df["signal"], "macd_signal"] = -1  # 空头信号

# 生成RSI信号
daily_df["rsi_signal"] = 0
daily_df.loc[daily_df["rsi"] < 30, "rsi_signal"] = 1  # 超卖，多头信号
daily_df.loc[daily_df["rsi"] > 70, "rsi_signal"] = -1  # 超买，空头信号

# 生成布林带信号
daily_df["bb_signal"] = 0
daily_df.loc[daily_df["close"] < daily_df["lower_band"], "bb_signal"] = 1  # 价格低于下轨，多头信号
daily_df.loc[daily_df["close"] > daily_df["upper_band"], "bb_signal"] = -1  # 价格高于上轨，空头信号

# 显示信号
daily_df[["close", "ma_cross_signal", "macd_signal", "rsi_signal", "bb_signal"]].tail(10)

## 9. 可视化交易信号

可视化交易信号，帮助分析策略的有效性。

In [ ]:
# 绘制价格和交易信号
plt.figure(figsize=(12, 8))

# 绘制价格
plt.plot(daily_df.index, daily_df["close"], label="Close")

# 绘制MA5和MA20
plt.plot(daily_df.index, daily_df["ma5"], label="MA5", alpha=0.7)
plt.plot(daily_df.index, daily_df["ma20"], label="MA20", alpha=0.7)

# 绘制多头信号
buy_signals = daily_df[daily_df["ma_cross_signal"] == 1]
plt.scatter(buy_signals.index, buy_signals["close"], marker="^", color="green", s=100, label="Buy Signal")

# 绘制空头信号
sell_signals = daily_df[daily_df["ma_cross_signal"] == -1]
plt.scatter(sell_signals.index, sell_signals["close"], marker="v", color="red", s=100, label="Sell Signal")

plt.title(f"{symbol} Price with MA Cross Signals")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

## 10. 策略回测

基于生成的交易信号进行简单的回测，评估策略性能。

In [ ]:
# 基于MA交叉信号进行回测
def backtest_ma_cross_strategy(df):
    df = df.copy()
    
    # 初始资金
    initial_capital = 10000.0
    position = 0
    capital = initial_capital
    df["capital"] = initial_capital
    
    # 遍历数据
    for i in range(1, len(df)):
        # 获取当前信号
        signal = df["ma_cross_signal"].iloc[i]
        price = df["close"].iloc[i]
        prev_price = df["close"].iloc[i-1]
        
        # 更新资金
        if position != 0:
            # 持有仓位，更新资金
            capital = capital + position * (price - prev_price)
        
        # 交易信号
        if signal == 1 and position <= 0:  # 多头信号
            position = capital / price  # 全仓买入
        elif signal == -1 and position >= 0:  # 空头信号
            position = -capital / price  # 全仓卖出
        
        # 记录资金
        df["capital"].iloc[i] = capital
    
    # 计算收益率
    df["returns"] = df["capital"].pct_change()
    
    # 计算累计收益率
    df["cumulative_returns"] = (1 + df["returns"]).cumprod() - 1
    
    # 计算回撤
    df["peak"] = df["capital"].cummax()
    df["drawdown"] = (df["capital"] - df["peak"]) / df["peak"]
    
    return df

# 运行回测
backtest_df = backtest_ma_cross_strategy(daily_df)

# 计算性能指标
total_return = (backtest_df["capital"].iloc[-1] / initial_capital) - 1
annual_return = (1 + total_return) ** (252 / len(backtest_df)) - 1
max_drawdown = backtest_df["drawdown"].min()
sharpe_ratio = backtest_df["returns"].mean() / backtest_df["returns"].std() * np.sqrt(252)

print(f"总收益率: {total_return:.2%}")
print(f"年化收益率: {annual_return:.2%}")
print(f"最大回撤: {max_drawdown:.2%}")
print(f"夏普比率: {sharpe_ratio:.2f}")

In [ ]:
# 绘制资金曲线
plt.figure(figsize=(12, 8))

# 绘制资金曲线
plt.subplot(2, 1, 1)
plt.plot(backtest_df.index, backtest_df["capital"], label="Capital")
plt.title(f"{symbol} MA Cross Strategy - Capital Curve")
plt.xlabel("Date")
plt.ylabel("Capital")
plt.legend()
plt.grid(True)

# 绘制回撤
plt.subplot(2, 1, 2)
plt.fill_between(backtest_df.index, 0, backtest_df["drawdown"] * 100, color="red", alpha=0.5)
plt.title("Drawdown (%)")
plt.xlabel("Date")
plt.ylabel("Drawdown (%)")
plt.grid(True)

plt.tight_layout()
plt.show()

## 11. 总结

通过这个示例，我们展示了如何使用Jupyter Notebook进行数据分析、技术指标计算、交易信号生成和策略回测。这些技术可以帮助您开发和优化自己的交易策略。